In [31]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file makes masks of the MagE slit projections
# and the two largest arcs of the Sunburst Arc
# matching the WCS of the v5 reductions of the HST
# observations of the Sunburst Arc (HST PIDs 15101,
# 15949).

In [32]:
import os

import numpy as np

from astropy.io import fits

import astropy.units as u

from astropy import wcs
from astropy.coordinates import SkyCoord
from regions import RectangleSkyRegion
from reproject.interpolation import reproject_interp

In [33]:
def make():

    home = os.getcwd()

    data = f'{home}/data'
    results = f'{home}/results'

    # File path to a v5 reduced HST observation. We just need this for the WCS, so it shouldn't matter
    # which v5 file we grab.
    file = f'{data}/hst/V5.0_PSZ1G311.65-18.48_F814W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'

    slits = {
        'm5' : ['15:50:01.1649 -78:11:07.822',0.850,10.000,313],
        'm4' : ['15:50:04.9279 -78:10:59.032',0.850,10.000,0],
        'm6' : ['15:50:06.6389 -78:10:57.412',0.850,10.000,12],
        'm3' : ['15:50:00.6009 -78:11:09.912',0.850,10.000,298],
        'm0' : ['15:50:04.4759 -78:10:59.652',1.0,10.000,349],
        'm2' : ['15:49:59.7480 -78:11:13.482',0.850,10.000,52],
        'm7' : ['15:50:07.3959 -78:10:56.962',0.850,10.000,0],
        'm8' : ['15:49:59.9499 -78:11:12.242',0.850,10.000,330],
        'm9' : ['15:50:00.3719 -78:11:10.512',0.850,10.000,302]
    }

    # Get the slits as Region objects
    #regions = Regions.read(f'{home}/Region Images/master_mage_slits_OBSERVED_V2redux.reg', format='ds9')
    regions = []

    for i, key in enumerate(slits):

        coords = SkyCoord(slits[key][0], unit=(u.hourangle, u.deg), frame='fk5')
        region = RectangleSkyRegion(center=coords, width=slits[key][1] * u.arcsecond, height=10 * u.arcsecond, angle=slits[key][3] * u.deg)
        regions.append(region)

    # Labels of the slits
    names = np.array(['M5', 'M4', 'M6', 'M3', 'M0', 'M2', 'M7', 'M8', 'M9'], dtype=str)

    # Get the header of the reference file
    header = fits.getheader(file)

    # Extract the WCS from the header
    w = wcs.WCS(header)

    # Make a header from the WCS information
    new_header = w.to_header()

    # For each slit
    for i, region in enumerate(regions):
        
        # Get the header of the reference file
        #header = fits.getheader(file)

        # Extract the WCS from the header
        #w = wcs.WCS(header)

        # Get the slit in pixel coordinates using the WCS
        pixel_region = region.to_pixel(w)

        # Make a mask from the pixel region
        mask = pixel_region.to_mask().to_image((header['NAXIS2'], header['NAXIS1']))

        # Make a header from the WCS information
        #new_header = w.to_header()

        # Save the mask with the basic WCS-only header
        fits.writeto(f'{results}/{names[i]}_mask_v5.fits', mask, header=new_header, overwrite=True)

    # The file path to the mask of the two largest arcs in the v4 HST reduction WCS
    arc_mask_file = f'{data}/hst/masks/arcMask.fits'

    # Reproject and interpolate the mask to the v5 HST reduction WCS
    arc_mask, _ = reproject_interp(arc_mask_file, header, hdu_in=0)
    
    # Where the interpolation produces values 0 - 1, reassign them as 1 to maintain
    # binary structure
    arc_mask = np.where(arc_mask > 0, 1, 0)

    # Create a new header from the v5 HST reduction WCS
    header = wcs.WCS(header).to_header()

    # Insert new keycards in the header to add additional information
    header.insert(1, ('CREATOR', 'M. Riley Owens', 'Creator of this file'))
    header.insert(2, ('EMAIL', 'm.riley.owens@gmail.com', 'Email of the creator of this file'))
    header['GENFILE'] = ('masks.ipynb', 'The file used to create this file')
    header['FILENAME'] = (f'arc_mask_v5.fits', 'Name of file')
    header['DESC'] = ('This file is a binary mask of the Sunburst Arc, aligned to the WCS of the \
        v5 reduction of the HST observations. The original mask was made for the VLT/MUSE footprint, and \
        then reprojected to the v4 HST reduction WCS. This file was created from the mask reprojected to \
        the v4 HST reduction WCS.', 'Description of file')

    # Save the new arc mask
    hdu = fits.PrimaryHDU(data=arc_mask, header=header)
    hdul = fits.HDUList([hdu])
    hdul.writeto(f'{results}/arc_mask_v5.fits', overwrite=True)

In [34]:
make()